In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
# Agregar la carpeta 'src' al sistema de rutas
sys.path.append(os.path.abspath('../src'))
# Ahora puedes importar el archivo o los módulos
from pricing_tools import *
import scipy.stats as stats
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import statsmodels.api as sm

### Leemos el diccionario de variables

In [ ]:
dict_path=r'../src/data_dict.xlsx'
dict = pd.read_excel(dict_path)
dict.head()

### Leemos el dataframe

In [ ]:
df_path=r'../src/AGUAACAGBC'
df=pd.read_parquet(df_path)
df.head()

In [ ]:
#Por si llegase a ser necesario reducir el tamaño del df para el analisis
df_muestra = df.sample(n=10000)

In [ ]:
###Definimos las variables de respuesta y peso
var_resp='stro_Corr_AGUAACAGBC'
peso='exp_corr_ACAGBC'
ano='year'

In [ ]:
df_muestra=df_muestra[df_muestra[peso]>0] #Necesitamos que la poliza tenga exposición

In [ ]:
df_muestra['resp']=df_muestra[var_resp]/df_muestra[peso]

In [ ]:
def graficar_histograma(df, columna, bins=10, titulo="Histograma", xlabel="Valores", ylabel="Frecuencia"):
    """
    Grafica un histograma de una variable específica de un DataFrame.
    
    Parámetros:
    - df: DataFrame de pandas que contiene los datos.
    - columna: Nombre de la columna que se desea graficar (str).
    - bins: Número de bins en el histograma (opcional, por defecto 10).
    - titulo: Título del gráfico (opcional).
    - xlabel: Etiqueta para el eje X (opcional).
    - ylabel: Etiqueta para el eje Y (opcional).
    """
    # Comprobamos si la columna existe en el DataFrame
    if columna not in df.columns:
        print(f"La columna '{columna}' no se encuentra en el DataFrame.")
        return
    
    # Graficar el histograma
    plt.figure(figsize=(8, 6))
    plt.hist(df[columna].dropna(), bins=bins, color="skyblue", edgecolor="black")
    plt.title(titulo)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.show()

In [ ]:
#Estudiamos la composicion de la cartera por años
df_muestra[ano]=df_muestra[ano].astype(str)
df_resumido = df_muestra.groupby(ano).agg({peso: 'sum', 'resp': 'mean'}).reset_index()
graficar_barras_linea(df_resumido,ano,peso,'resp')

In [ ]:
### Ejemplo variable importante 
### Antiguedad del edificio
var='antigedif'
df_muestra[var].dtype
df_resumido = df_muestra.groupby(var).agg({peso: 'sum', 'resp': 'mean'}).reset_index()
graficar_barras_linea(df_resumido,var,peso,'resp')

In [ ]:
antig_corregida=generar_formato_exposicion_homogenea(df_muestra, 'antigedif', peso, 10)
df_muestra['antigedif_2']=antig_corregida #Sobreescribimos la columna corregida

In [ ]:
df_muestra['antigedif_2']=df_muestra['antigedif_2'].astype(str)
df_resumido = df_muestra.groupby('antigedif_2').agg({peso: 'sum', 'resp': 'mean'}).reset_index()
graficar_barras_linea(df_resumido,'antigedif_2',peso,'resp')

In [ ]:
### Y que hacemos con los Nan? -> por ejemplo convertirlos en la moda de la variable
df_muestra['antigedif_2'].replace('nan',np.nan).mode(dropna=True)[0]

In [ ]:
### Ahora sustituimos los nan por el valor moda
df_muestra['antigedif_2']=df_muestra['antigedif_2'].replace('nan','(51.0, 56.0]')
df_resumido = df_muestra.groupby('antigedif_2').agg({peso: 'sum', 'resp': 'mean'}).reset_index()
graficar_barras_linea(df_resumido,'antigedif_2',peso,'resp')

In [ ]:
factores=['year','antigedif_2','anualidad_seguro','FP','K_ACAGBC','SUPERFICIE','stro_1a_AGUAACAGBC','NIF_TIPO']
df_muestra[factores].dtypes

In [ ]:
# Creamos variables dummy para las columnas categóricas
key_words=['year', 'antigedif_2','NIF_TIPO']
df_encoded = pd.get_dummies(df_muestra, columns=key_words, drop_first=True)

In [ ]:
cols_encoded = [each for each in df_encoded.columns if any(keyword in each for keyword in key_words)]
cols_encoded

In [ ]:
factores_updated=[each for each in factores if each not in key_words]
factores_updated

In [ ]:
list(set(cols_encoded+factores_updated))

In [ ]:
# Prepara los datos
df_encoded = df_encoded[list(set(cols_encoded+factores_updated))].astype('float')
X=df_encoded# Selecciona las variables predictoras
X = sm.add_constant(X)            # Agrega una constante al modelo (intercepto)
y = df_muestra['resp']                   # Variable de respuesta

# Definir la exposición como un offset en logaritmo
offset = np.log(df_muestra[peso])

# Crear el modelo GLM con enlace logarítmico y familia Poisson
modelo = sm.GLM(y, X, family=sm.families.Poisson(), offset=offset)

# Ajustar el modelo
resultado = modelo.fit()

# Mostrar los resultados del modelo
print(resultado.summary())

In [ ]:
def calculate_bic(model):
    n = len(model.model.endog)  # Número de observaciones
    k = model.df_model + 1  # Número de parámetros (df_model incluye los coeficientes, sumamos 1 por el intercepto)
    bic = np.log(n) * k - 2 * model.llf  # BIC = ln(n) * k - 2 * log-verosimilitud
    return bic
calculate_bic(resultado)

In [ ]:
factores=['year','antigedif_2','anualidad_seguro','FP','K_ACAGBC','SUPERFICIE','stro_1a_AGUAACAGBC']
df_muestra[factores].dtypes

In [ ]:
# Creamos variables dummy para las columnas categóricas
key_words=['year', 'antigedif_2']
df_encoded = pd.get_dummies(df_muestra, columns=key_words, drop_first=True)

In [ ]:
cols_encoded = [each for each in df_encoded.columns if any(keyword in each for keyword in key_words)]
cols_encoded

In [ ]:
factores_updated=[each for each in factores if each not in key_words]
factores_updated

In [ ]:
# Prepara los datos
df_encoded = df_encoded[list(set(cols_encoded+factores_updated))].astype('float')
X=df_encoded# Selecciona las variables predictoras
X = sm.add_constant(X)            # Agrega una constante al modelo (intercepto)
y = df_muestra['resp']                   # Variable de respuesta

# Definir la exposición como un offset en logaritmo
offset = np.log(df_muestra[peso])

# Crear el modelo GLM con enlace logarítmico y familia Poisson
modelo1 = sm.GLM(y, X, family=sm.families.Poisson(), offset=offset)

# Ajustar el modelo
resultado1 = modelo1.fit()

# Mostrar los resultados del modelo
print(resultado.summary())

In [ ]:
calculate_bic(resultado1)